# Tests for Diag 

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
import yt
##yt.mylog.setLevel()
yt.utilities.logger.disable_stream_logging()

/Users/tflv6/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
dataDir = '../single_level/'
baseName = 'plt'

In [70]:
ds = yt.load(dataDir + baseName + '0000001')

In [121]:
ds.field_list

[('boxlib', 'Hnuc'),
 ('boxlib', 'MachNumber'),
 ('boxlib', 'Pi'),
 ('boxlib', 'S'),
 ('boxlib', 'X(A)'),
 ('boxlib', 'abar'),
 ('boxlib', 'conductivity'),
 ('boxlib', 'deltaT'),
 ('boxlib', 'deltagamma'),
 ('boxlib', 'deltap'),
 ('boxlib', 'divw0'),
 ('boxlib', 'entropy'),
 ('boxlib', 'entropypert'),
 ('boxlib', 'gpix'),
 ('boxlib', 'gpiy'),
 ('boxlib', 'h'),
 ('boxlib', 'h0'),
 ('boxlib', 'magvel'),
 ('boxlib', 'momentum'),
 ('boxlib', 'omegadot(A)'),
 ('boxlib', 'p0'),
 ('boxlib', 'p0pluspi'),
 ('boxlib', 'pioverp0'),
 ('boxlib', 'rho'),
 ('boxlib', 'rho0'),
 ('boxlib', 'rhoX(A)'),
 ('boxlib', 'rhoh'),
 ('boxlib', 'rhoh0'),
 ('boxlib', 'rhohpert'),
 ('boxlib', 'rhopert'),
 ('boxlib', 'tfromh'),
 ('boxlib', 'tfromp'),
 ('boxlib', 'thermal'),
 ('boxlib', 'tpert'),
 ('boxlib', 'velx'),
 ('boxlib', 'vely'),
 ('boxlib', 'vort'),
 ('boxlib', 'w0x'),
 ('boxlib', 'w0y')]

In [89]:
dx, dy, dz = ds.domain_width / ds.domain_dimensions # think this is ok for single level / UG

## diag_vel.out

Look at the file (assuming its only a few rows)

In [111]:
cat ../diag_vel.out

time                max{U}              max{Mach}           tot kin energy      tot grav energy     tot int energy      dt                  
0.0000000000e+00    0.0000000000e+00    0.0000000000e+00    0.0000000000e+00    -1.9021530793e+37   9.4816020347e+30    1.8387268888e-01    
1.8387268888e-01    6.9877161823e+02    9.1367124016e-05    2.0362137489e+22    -1.9021530572e+37   9.4816020228e+30    1.8387268888e-01    


Load up data needed for velocities and kinetic energy calculation

In [72]:
all_data_level_0 = ds.covering_grid(level=0, left_edge=[0,0.0,0.0], dims=[64, 64, 1])
vx = all_data_level_0['velx']
vx = vx[:,:,0].transpose()
vy = all_data_level_0['vely']
vy = vy[:,:,0].transpose()
rho = all_data_level_0['rho']
rho = rho[:,:,0].transpose()
w0y = all_data_level_0['w0y']
w0y = w0y[:,:,0].transpose()
vel = np.sqrt(vx**2 + (vy+w0y)**2)

Whats the maximum velocity according to the plotfile data?

In [113]:
np.max(vel) 

698.7716182269953 dimensionless

Whats the total kinetic energy according to the data in the plotfile? The codes diag file currently ignores the 0.5, and will "Normalise"?!? for dL, dA, or dV

In [119]:
ke = 0.5 * rho * vel**2
np.sum(2. * ke *dx * dy)

2.036213748909617e+22 code_length**2

Whats the total internal energy according to the data in the plotfile? I had to look inside the EOS code to see how this is calculated

In [220]:
T = all_data_level_0['tfromh']
T = T[:,:,0].transpose()

In [224]:
k_B = 1.3806488e-16   # erg/K
n_A = 6.02214129e23  # mol^-1
R = k_B * n_A

mu = 1

gamma = 5./3.
cv = R /mu / (gamma-1)
eint = cv * T 

In [225]:
np.sum(eint*rho * dx**2)

9.481602022761284e+30 code_length**2

Whats the maximum mach number?

In [232]:
rho = all_data_level_0['rho']
rho = rho[:,:,0].transpose()
p = all_data_level_0['p0']
p = p[:,:,0].transpose()
cs = np.sqrt(gamma * p / rho)
mach = vel / cs

print( np.max(mach))

9.136712405363016e-05 dimensionless


Whats the gravitational potential energy? **DEFINITELY DONT HAVE THIS RIGHT FOR PLANAR! **

## diag_temp.out

In [234]:
cat ../diag_temp.out

time                max{T}              x(max{T})           y(max{T})           z(max{T})           vx(max{T})          vy(max{T})          vz(max{T})          
0.0000000000e+00    4.2209256938e+05    7.8125000000e+06    7.5781250000e+08    0.0000000000e+00    0.0000000000e+00    0.0000000000e+00    0.0000000000e+00    
1.8387268888e-01    4.2209255760e+05    5.0781250000e+08    7.5781250000e+08    0.0000000000e+00    2.1473842671e+00    6.9876831867e+02    0.0000000000e+00    


In [237]:
T = all_data_level_0['tfromh']
T = T[:,:,0].transpose()
print(np.max(T))

422092.5576018601 dimensionless


can have a look at the actual coordinates, but its tricky if there are multiple maxima - might want to setup a test where one cell is very hot?

In [287]:
iy,ix = np.where( T == np.max(T))

In [288]:
T[iy, ix]

YTArray([422092.55760186]) (dimensionless)

In [289]:
ix * dx + dx/2

[5.078125e+08] code_length

In [290]:
iy * dy + dy/2

[7.578125e+08] code_length

In [293]:
vx[iy,ix]

YTArray([2.14738427]) (dimensionless)

In [296]:
vy[iy, ix] + w0y[iy,ix]

[698.76831867] dimensionless

## diag_enuc.out

Dont really have a good test for this. But since its so similar logically to temp, i think im willing to take it as OK for now (even to the point of making a PR). If there is a bug - it will get noticed later